In [1]:
import torch
import torch.nn as nn

0.00026722238908405416
0.0002229883975815028
0.00022028142120689154
0.0002745842284639366
0.00023446396517101675
0.00022384159456123598
0.00024397196466452442
0.00024525218759663403
0.00023349763432634063
0.00024575241332058795
0.00022118984270491637
0.0002341404433536809
0.0002261278932564892
0.00023443707686965354
0.0002581316657597199
0.00024152823971235193
0.0002164314573747106
0.00021622630811179988
0.00023868255084380507


In [2]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2-xl')
model = GPT2LMHeadModel.from_pretrained('gpt2-xl')

In [3]:
text = "Marry had a little"
encoded_input = tokenizer.batch_encode_plus([text], return_tensors='pt')
output = model(**encoded_input)

In [4]:
predictions = output.logits

# get the predicted next sub-word (in our case, the word 'man')
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode([predicted_index])

In [5]:
predicted_text

' bit'

In [6]:
import datasets

dataset = datasets.load_dataset('wikipedia', '20220301.simple')

Found cached dataset wikipedia (/home/ubuntu/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(50257, 512)
        
        self.transformer = nn.Transformer(num_encoder_layers=3, num_decoder_layers=3, d_model=512, nhead=8, dim_feedforward=2048)
        self.out_embedding = nn.Linear(512, 50257)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(src=x, tgt=torch.stack((x[1:x.size(1)], [x[:,0,:]])))
        x = self.out_embedding(x)
        x = torch.softmax(x, dim=2)
        return x

    def _init_weights(self):
        nn.init.normal_(self.embedding.weight, std=0.02)
        nn.init.normal_(self.out_embedding.weight, std=0.02)

In [22]:
from torch.optim import AdamW
my_model = Model()

In [23]:
import random
my_model.train()
optimizer = AdamW(my_model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

total_loss = 0

for parameter in model.parameters():
    parameter.requires_grad = False

for i, item in enumerate(dataset['train']):
    if i >= 200:
        break
    k = 2
    text = tokenizer.encode_plus(item['text'], max_length=256, truncation=True)['input_ids']
    batch = torch.tensor([list(text[i:i+k]) for i in range(0, 1 * k, k)])
    output = my_model(batch)
    with torch.no_grad():
        output_gpt2 = model(input_ids=batch).logits
    loss = criterion(output_gpt2, output)
    total_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    if i % 5 == 0 and i > 0:
        print(total_loss)
        total_loss = 0



TypeError: expected Tensor as element 1 in argument 0, but got list

In [14]:
my_model.eval()
encoded_input = tokenizer.batch_encode_plus(['dim dim'], return_tensors='pt')
print(tokenizer.decode(my_model(encoded_input['input_ids']).argmax(dim=2)[0]))

 mic dim
